# Sync local data


In [1]:
# sync data from google drive (first time takes up to 2 hours ;)
!rclone sync vfa-Measurements:/ ../data/Measurements/

2018/09/28 08:17:51 NOTICE: byspireMonitoring28_12_1402.csv: Duplicate object found in source - ignoring
2018/09/28 08:17:51 NOTICE: Rack_2_3_05_1559.csv: Duplicate object found in source - ignoring
2018/09/28 08:17:51 NOTICE: Rack_2_4_05_1059.csv: Duplicate object found in source - ignoring
2018/09/28 08:17:51 NOTICE: Rack_3_3_05_1558.csv: Duplicate object found in source - ignoring
2018/09/28 08:17:51 NOTICE: Rack_3_4_05_1057.csv: Duplicate object found in source - ignoring


# Parse the local data

In [1]:
import pandas as pd
import sys

sys.path.append("../shared-libraries/")
pd.set_option('display.max_rows', 10)

In [2]:
files = !ls ../data/Measurements/
se_files = pd.Series(files)
se_files

0                  Rack_0_07_06-2108.csv
1                  Rack_0_07_06-2109.csv
2                  Rack_0_07_09-1431.csv
3                  Rack_0_08_05-1523.csv
4                  Rack_0_08_08-1743.csv
                      ...               
6222    byspireMonitoring_21_03-1124.csv
6223    byspireMonitoring_21_03-1243.csv
6224    byspireMonitoring_21_03-1319.csv
6225    byspireMonitoring_22_03-1129.csv
6226    byspireMonitoring_22_03-1143.csv
Length: 6227, dtype: object

In [3]:
# Check the first few characters for different types of files
se_files.str.slice(0,4).value_counts()

Rack    5676
bysp     551
dtype: int64

In [4]:
from data_io import parse_sensor_filename
se_files.apply(parse_sensor_filename)

0       {'FILENAME': 'Rack_0_07_06-2108.csv', 'RACK': ...
1       {'FILENAME': 'Rack_0_07_06-2109.csv', 'RACK': ...
2       {'FILENAME': 'Rack_0_07_09-1431.csv', 'RACK': ...
3       {'FILENAME': 'Rack_0_08_05-1523.csv', 'RACK': ...
4       {'FILENAME': 'Rack_0_08_08-1743.csv', 'RACK': ...
                              ...                        
6222    {'FILENAME': 'byspireMonitoring_21_03-1124.csv...
6223    {'FILENAME': 'byspireMonitoring_21_03-1243.csv...
6224    {'FILENAME': 'byspireMonitoring_21_03-1319.csv...
6225    {'FILENAME': 'byspireMonitoring_22_03-1129.csv...
6226    {'FILENAME': 'byspireMonitoring_22_03-1143.csv...
Length: 6227, dtype: object

In [59]:
%run -i ../shared-libraries/data_io.py

In [60]:
df_inventory = se_files.apply(parse_sensor_filename).apply(pd.Series)
df_inventory

,FILENAME,FILE_TIMESTAMP,PATTERN_NAME,RACK
0,Rack_0_07_06-2108.csv,2018-06-07 21:08:00,1,0
1,Rack_0_07_06-2109.csv,2018-06-07 21:09:00,1,0
2,Rack_0_07_09-1431.csv,2018-09-07 14:31:00,1,0
3,Rack_0_08_05-1523.csv,2018-05-08 15:23:00,1,0
4,Rack_0_08_08-1743.csv,2018-08-08 17:43:00,1,0
...,...,...,...,...
6222,byspireMonitoring_21_03-1124.csv,2018-03-21 11:24:00,2,None
6223,byspireMonitoring_21_03-1243.csv,2018-03-21 12:43:00,2,None
6224,byspireMonitoring_21_03-1319.csv,2018-03-21 13:19:00,2,None
6225,byspireMonitoring_22_03-1129.csv,2018-03-22 11:29:00,2,None


### Check for missed files

In [61]:
df_inventory['PATTERN_NAME'].value_counts(dropna=False)

1      5675
2       542
3         6
NaN       4
Name: PATTERN_NAME, dtype: int64

In [62]:
df_inventory[df_inventory['PATTERN_NAME'].isnull()]

,FILENAME,FILE_TIMESTAMP,PATTERN_NAME,RACK
16,Rack_1.0_22_03_1259.csv.xlsx,NaT,NaN,NaN
5736,byspireMonitoring16_01_0652.csv.xlsx,NaT,NaN,NaN
5738,byspireMonitoring16_01_0939.csv.xlsx,NaT,NaN,NaN
6203,byspireMonitoring_03_01-1355.xlsx,NaT,NaN,NaN


### Load up the files

In [63]:
def my_csv_reader(fn):
    resp = {'DF': None, 'DF_EXCEPTION': None}
    try:
        resp['DF'] = pd.read_csv("../data/Measurements/" + fn)
    except Exception as e:
        resp['DF_EXCEPTION'] = e
    return pd.Series(resp)

df_csv_reader = df_inventory[df_inventory.PATTERN_NAME.notnull()]\
    .FILENAME.apply(my_csv_reader)

df_inventory['DF'] = df_csv_reader['DF']
df_inventory['DF_EXCEPTION'] = df_csv_reader['DF_EXCEPTION']
df_inventory

,FILENAME,FILE_TIMESTAMP,PATTERN_NAME,RACK,DF,DF_EXCEPTION
0,Rack_0_07_06-2108.csv,2018-06-07 21:08:00,1,0,Date Time Air temp Humidity ...,None
1,Rack_0_07_06-2109.csv,2018-06-07 21:09:00,1,0,Date Time Air temp Humidity ...,None
2,Rack_0_07_09-1431.csv,2018-09-07 14:31:00,1,0,Date Time Air temp Humidity ...,None
3,Rack_0_08_05-1523.csv,2018-05-08 15:23:00,1,0,Date Time Air temp Humidity ...,None
4,Rack_0_08_08-1743.csv,2018-08-08 17:43:00,1,0,Date Time Air temp Humidity W...,None
...,...,...,...,...,...,...
6222,byspireMonitoring_21_03-1124.csv,2018-03-21 11:24:00,2,None,21/03/2018 11:24:45 23.58 27.53 20.0 1...,None
6223,byspireMonitoring_21_03-1243.csv,2018-03-21 12:43:00,2,None,21/03/2018 12:43:47 25.37 25.77 20.0 ...,None
6224,byspireMonitoring_21_03-1319.csv,2018-03-21 13:19:00,2,None,21/03/2018 13:20:08 27.48 39.64 109.0 ...,None
6225,byspireMonitoring_22_03-1129.csv,2018-03-22 11:29:00,2,None,22/03/2018 11:30:04 26.0 45.72 40.0 2...,None


In [64]:
df_inventory[df_inventory.DF_EXCEPTION.notnull()]

,FILENAME,FILE_TIMESTAMP,PATTERN_NAME,RACK,DF,DF_EXCEPTION
952,Rack_2_06_08-1045.csv,2018-08-06 10:45:00,1,2,None,No columns to parse from file
1528,Rack_2_17_04-1351.csv,2018-04-17 13:51:00,1,2,None,No columns to parse from file
2418,Rack_2_26_04_1358.csv,2018-04-26 13:58:00,1,2,None,No columns to parse from file
2502,Rack_2_27_04-1508.csv,2018-04-27 15:08:00,1,2,None,No columns to parse from file
2503,Rack_2_27_04-1509.csv,2018-04-27 15:09:00,1,2,None,No columns to parse from file
...,...,...,...,...,...,...
4699,Rack_4_09_08-1220.csv,2018-08-09 12:20:00,1,4,None,No columns to parse from file
4700,Rack_4_09_08-1221.csv,2018-08-09 12:21:00,1,4,None,No columns to parse from file
4701,Rack_4_09_08-1223.csv,2018-08-09 12:23:00,1,4,None,No columns to parse from file
4703,Rack_4_09_08-1232.csv,2018-08-09 12:32:00,1,4,None,No columns to parse from file


... work in progress